In [35]:
import sys
import pandas as pd
import numpy as np
import datetime as dt

# 프로젝트 루트 직접 지정
sys.path.insert(0, r"C:\airflow")

# 코드

In [36]:
sales_daily_orders_alerts = pd.read_csv(r"C:\Local_DB\영업관리부_DB\sales_daily_orders_alerts.csv", encoding="utf-8")
sales_daily_orders_alerts.head()

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,일별_전체_평균,vs일별전체_비율,fee_ad_배민,fee_ad_쿠팡,settlement_amount_배민,settlement_amount_쿠팡,total_amount_배민,total_amount_쿠팡,total_order_count_배민,total_order_count_쿠팡
0,2025-12-31,2025-12-31 18:48:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,6,171200.0,12910.0,28533.333333,NaN,...,1579543.0,10.8,0.0,12910.0,0.0,105517.0,0.0,171200.0,0,6
1,2026-01-01,2026-01-01 19:58:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,7,224300.0,18495.0,32042.857143,NaN,...,1843271.0,12.2,0.0,18495.0,0.0,143391.0,0.0,224300.0,0,7
2,2026-01-02,2026-01-02 21:20:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,5,95600.0,7200.0,19120.000000,NaN,...,1376062.0,6.9,0.0,7200.0,0.0,40216.0,0.0,95600.0,0,5
3,2026-01-03,2026-01-03 11:19:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,1,17600.0,0.0,17600.000000,NaN,...,1412462.0,1.2,0.0,0.0,0.0,7901.0,0.0,17600.0,0,1
4,2026-01-13,2026-01-13 21:13:23,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,22,546100.0,25345.0,24822.727273,NaN,...,1816329.0,30.1,4000.0,21345.0,198919.0,173831.0,279600.0,266500.0,13,9


# 우리가게now

In [37]:
now_df = pd.read_parquet(r"C:\Local_DB\temp\baemin_store_now_path_20260115.parquet")
now_df["collected_date"] = now_df["collected_at"].str[:10]
now_df.drop_duplicates(subset=['store_id', 'collected_date'], keep='last', inplace=True)
now_df["stores_name"] = now_df["store_name"].str.split(" ").str[-2:].str.join(" ")
col = ['collected_date', 'stores_name', '조리소요시간',
       '조리소요시간_순위비율', '주문접수시간', '주문접수시간_순위비율',
       '조리시간준수율', '조리시간준수율_순위비율', '주문접수율',
       '주문접수율_순위비율', '최근별점']

now_df = now_df[col]
now_df

,collected_date,stores_name,조리소요시간,조리소요시간_순위비율,주문접수시간,주문접수시간_순위비율,조리시간준수율,조리시간준수율_순위비율,주문접수율,주문접수율_순위비율,최근별점
1,2026-01-14,곱도리탕 교대점,21,0.5,13,0.5,0.97,0.6,1.000,0.1,5.0
2,2026-01-13,도리당 가락점,30,0.2,16,0.4,1.00,0.1,0.987,0.4,4.9


 # 변경이력

In [38]:
import pandas as pd
import numpy as np

In [39]:
history_df = pd.read_parquet(r"C:\Local_DB\temp\baemin_history_path_20260115.parquet")

In [40]:
history_df["change_date"] = history_df["변경시간"].str[:10]
history_df = history_df.drop_duplicates(subset=["변경시간", "store_id"], keep='last')
history_df["stores_name"] = history_df["매장명"].str.split(" ").str[-2:].str.join(" ")
history_trans_df = history_df[["change_date", "stores_name", "대분류"]]
history_trans_df = history_trans_df.groupby(["change_date", "stores_name", "대분류"]).size().reset_index(name='cnt').pivot_table(
    index=["change_date", "stores_name"],
    columns="대분류",
    values="cnt",
    fill_value=0,
    aggfunc=np.sum
).reset_index().rename_axis(None, axis=1)


C:\Users\민준\AppData\Local\Temp\ipykernel_14452\3342331168.py:5: FutureWarning: The provided callable <function sum at 0x00000208C9C168E0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  history_trans_df = history_trans_df.groupby(["change_date", "stores_name", "대분류"]).size().reset_index(name='cnt').pivot_table(


In [41]:
history_trans_df.head()

,change_date,stores_name,영업임시중지 변경,운영시간 변경,휴무일 변경
0,2025-09-08,도리당 평택서정점,0,1,0
1,2025-09-09,도리당 평택서정점,0,2,0
2,2025-09-10,도리당 평택서정점,0,2,0
3,2025-09-11,도리당 평택서정점,2,1,0
4,2025-09-12,도리당 평택서정점,6,1,0


In [42]:
# 고정 컬럼명 정의
target_categories = {
    "영업임시 중지": "영업임시중지_변경_cnt",
    "휴무일 변경": "휴무일변경_변경_cnt",
    "운영시간 변경": "운영시간변경_변경_cnt"
}

# 날짜/매장별 그룹화
history_pivot = history_df.groupby(["change_date", "stores_name"]).apply(
    lambda x: pd.Series({
        target_categories.get(cat, cat.replace(" ", "") + "_변경_cnt"): (x["대분류"] == cat).sum()
        for cat in x["대분류"].unique()
    })
).reset_index()


# 컬럼 순서 정리
base_cols = ["change_date", "stores_name"]
count_cols = [col for col in history_pivot.columns if col not in base_cols]
history_pivot = history_pivot[base_cols + sorted(count_cols, key=str)]

C:\Users\민준\AppData\Local\Temp\ipykernel_14452\1514524400.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  history_pivot = history_df.groupby(["change_date", "stores_name"]).apply(


In [43]:
history_pivot

,change_date,stores_name,0,level_2
0,2025-09-08,도리당 평택서정점,1,운영시간변경_변경_cnt
1,2025-09-09,도리당 평택서정점,2,운영시간변경_변경_cnt
2,2025-09-10,도리당 평택서정점,2,운영시간변경_변경_cnt
3,2025-09-11,도리당 평택서정점,2,영업임시중지변경_변경_cnt
4,2025-09-11,도리당 평택서정점,1,운영시간변경_변경_cnt
...,...,...,...,...
150,2026-01-09,도리당 평택서정점,2,운영시간변경_변경_cnt
151,2026-01-10,도리당 평택서정점,6,운영시간변경_변경_cnt
152,2026-01-11,도리당 평택서정점,1,운영시간변경_변경_cnt
153,2026-01-11,도리당 평택서정점,4,영업임시중지변경_변경_cnt


# 토더

In [ ]:
import os

In [ ]:
toorder_review_df = pd.read_excel(r"E:\down\toorder_review_doridang1_2025-12-02.xlsx", header=3)

In [5]:
toorder_review_df = pd.read_parquet(r"C:\Doridang\temp\toorder_review_path_20260115.parquet")
toorder_review_df

,매장명,매장명.1,매장 태그,오픈일,채널,주문 수,리뷰 수,답변완료 수,평균 별점,date
0,일산백석점,None,None,NaN,None,14,2,2,5.0,2025-12-01
1,None,일산백석점,None,NaN,배달의민족,7,2,2,5.0,2025-12-01
2,None,일산백석점,None,NaN,쿠팡이츠,7,0,0,0.0,2025-12-01
3,None,일산백석점,None,NaN,요기요,0,0,0,0.0,2025-12-01
4,None,일산백석점,None,NaN,땡겨요,0,0,0,0.0,2025-12-01
...,...,...,...,...,...,...,...,...,...,...
24520,None,대전관평점,None,NaN,땡겨요,0,0,0,0.0,2026-01-14
24521,None,대전관평점,None,NaN,먹깨비,0,0,0,0.0,2026-01-14
24522,None,대전관평점,None,NaN,네이버 지도,0,0,0,0.0,2026-01-14
24523,None,대전관평점,None,NaN,구글 지도,0,0,0,0.0,2026-01-14


In [ ]:
file_name = os.path.basename(r"E:\down\toorder_review_doridang1_2025-12-02.xlsx")
toorder_review_df["date"] = file_name.split("_")[-1].split(".")[0]

In [ ]:
col = ['date', '매장명.1', '채널', '주문 수', '리뷰 수', '답변완료 수', '평균 별점',
       ]
toorder_review_df = toorder_review_df[col]
toorder_review_df = toorder_review_df[~toorder_review_df["채널"].isnull()]
toorder_review_df = toorder_review_df.groupby(["date", "매장명.1"]).agg(전체_주문수=("주문 수", "sum"),
                                                  전체_리뷰수=("리뷰 수", "sum"),
                                                전체_답변완료수=("답변완료 수", "sum"),
                                                전체_평균별점=("평균 별점", "mean")
).reset_index()

toorder_review_df.rename(columns={"매장명.1": "stores_name"}, inplace=True)





toorder_review_df["stores_name"] = "도리당 " + toorder_review_df["stores_name"]

toorder_review_df["stores_name"] = toorder_review_df["stores_name"].replace({"도리당 일산백석점": "도리당 백석점",
                                              "도리당 서울대점" : "도리당 서울대점",
                                              "도리당 구로디지털단지점" : "도리당 구로디지털점",
                                              "도리당 충주봉방점" : "도리당 충주역점"
                                              })

In [6]:
pd.read_parquet(r"C:\Local_DB\temp\preprocessed_toorder_review_path_20260115.parquet")

,date,stores_name,전체_주문수,전체_리뷰수,전체_답변완료수,전체_평균별점
0,2025-12-01,도리당 가락점,0,2,2,0.714286
1,2025-12-01,도리당 강동점,48,22,22,2.142857
2,2025-12-01,도리당 경북상주점,6,0,0,0.000000
3,2025-12-01,도리당 공덕점,19,3,3,1.428571
4,2025-12-01,도리당 광명철산점,0,0,0,0.000000
...,...,...,...,...,...,...
3055,2026-01-14,도리당 평택비전점,19,5,0,2.071429
3056,2026-01-14,도리당 평택서정점,55,6,0,1.428571
3057,2026-01-14,도리당 하늘도시점,56,8,0,1.428571
3058,2026-01-14,도리당 행신점,7,1,0,0.714286


# 병합

In [45]:
sales_daily_orders_alerts

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,일별_전체_평균,vs일별전체_비율,fee_ad_배민,fee_ad_쿠팡,settlement_amount_배민,settlement_amount_쿠팡,total_amount_배민,total_amount_쿠팡,total_order_count_배민,total_order_count_쿠팡
0,2025-12-31,2025-12-31 18:48:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,6,171200.0,12910.0,28533.333333,NaN,...,1579543.0,10.8,0.0,12910.0,0.0,105517.0,0.0,171200.0,0,6
1,2026-01-01,2026-01-01 19:58:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,7,224300.0,18495.0,32042.857143,NaN,...,1843271.0,12.2,0.0,18495.0,0.0,143391.0,0.0,224300.0,0,7
2,2026-01-02,2026-01-02 21:20:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,5,95600.0,7200.0,19120.000000,NaN,...,1376062.0,6.9,0.0,7200.0,0.0,40216.0,0.0,95600.0,0,5
3,2026-01-03,2026-01-03 11:19:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,1,17600.0,0.0,17600.000000,NaN,...,1412462.0,1.2,0.0,0.0,0.0,7901.0,0.0,17600.0,0,1
4,2026-01-13,2026-01-13 21:13:23,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,22,546100.0,25345.0,24822.727273,NaN,...,1816329.0,30.1,4000.0,21345.0,198919.0,173831.0,279600.0,266500.0,13,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,2026-01-10,2026-01-10 23:56:23,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,64,2164800.0,41660.0,33825.000000,1757957.14,...,2222743.0,97.4,0.0,41660.0,1213095.0,379310.0,1590300.0,574500.0,48,16
294,2026-01-11,2026-01-11 23:03:33,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,99,3105200.0,41910.0,31365.656566,1844992.86,...,2451950.0,126.6,0.0,41910.0,1987830.0,423407.0,2446100.0,659100.0,77,22
295,2026-01-12,2026-01-12 23:23:54,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,41,1294900.0,12070.0,31582.926829,1867800.00,...,2019570.0,64.1,0.0,12070.0,919807.0,113090.0,1119300.0,175600.0,36,5
296,2026-01-13,2026-01-13 23:46:11,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,61,1767600.0,24720.0,28977.049180,1895492.86,...,1816329.0,97.3,0.0,24720.0,1125162.0,193171.0,1442600.0,325000.0,48,13


In [ ]:
merged_df = sales_daily_orders_alerts.merge(
    now_df,
    how="left",
    left_on=["order_daily", "매장명"],
    right_on=["collected_date", "stores_name"]
)
merged_df.drop(columns=["collected_date", "stores_name"], inplace=True)
merged_df.head()

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,total_order_count_쿠팡,조리소요시간,조리소요시간_순위비율,주문접수시간,주문접수시간_순위비율,조리시간준수율,조리시간준수율_순위비율,주문접수율,주문접수율_순위비율,최근별점
0,2025-12-01,2025-12-01 20:21:00,도리당 교대점,심성준 이사,simjeong00@kakao.com,12,327100.0,15540.0,27258.333333,NaN,...,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-12-01,2025-12-01 23:42:37,도리당 미사점,김대진 팀장,sanbogaja81@kakao.com,63,4492100.0,163890.0,71303.174603,NaN,...,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-12-01,2025-12-01 23:43:21,도리당 송파점,이병두 과장,byoungd201@kakao.com,53,1380300.0,72010.0,26043.396226,NaN,...,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-12-01,2025-12-01 23:45:38,도리당 운정점,김덕기 과장,kdk1402@kakao.com,39,1106700.0,42840.0,28376.923077,NaN,...,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-12-01,2025-12-01 23:33:00,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,48,1213600.0,23670.0,25283.333333,NaN,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
pd.read_parquet(r"C:\Local_DB\temp\joined_orders_now_path_20260115.parquet").head()

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,stores_name,조리소요시간,조리소요시간_순위비율,주문접수시간,주문접수시간_순위비율,조리시간준수율,조리시간준수율_순위비율,주문접수율,주문접수율_순위비율,최근별점
0,2025-12-31,2025-12-31 18:48:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,6,171200.0,12910.0,28533.333333,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2026-01-01,2026-01-01 19:58:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,7,224300.0,18495.0,32042.857143,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-01-02,2026-01-02 21:20:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,5,95600.0,7200.0,19120.000000,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-01-03,2026-01-03 11:19:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,1,17600.0,0.0,17600.000000,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2026-01-13,2026-01-13 21:13:23,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,22,546100.0,25345.0,24822.727273,NaN,...,도리당 가락점,30.0,0.2,16.0,0.4,1.0,0.1,0.987,0.4,4.9


In [ ]:
merged_df = merged_df.merge(history_pivot,
                how="left",
                left_on=["order_daily", "매장명"],
                right_on=["change_date", "stores_name"]
)
merged_df.drop(columns=["change_date", "stores_name"], inplace=True)
merged_df

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,조리소요시간,조리소요시간_순위비율,주문접수시간,주문접수시간_순위비율,조리시간준수율,조리시간준수율_순위비율,주문접수율,주문접수율_순위비율,최근별점,영업임시중지변경_변경_cnt
0,2025-12-01,2025-12-01 20:21:00,도리당 교대점,심성준 이사,simjeong00@kakao.com,12,327100.0,15540.0,27258.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-12-01,2025-12-01 23:42:37,도리당 미사점,김대진 팀장,sanbogaja81@kakao.com,63,4492100.0,163890.0,71303.174603,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-12-01,2025-12-01 23:43:21,도리당 송파점,이병두 과장,byoungd201@kakao.com,53,1380300.0,72010.0,26043.396226,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-12-01,2025-12-01 23:45:38,도리당 운정점,김덕기 과장,kdk1402@kakao.com,39,1106700.0,42840.0,28376.923077,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-12-01,2025-12-01 23:33:00,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,48,1213600.0,23670.0,25283.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2026-01-13,2026-01-13 23:45:47,도리당 미사점,김대진 팀장,sanbogaja81@kakao.com,65,1731600.0,66852.0,26640.000000,2343242.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,2026-01-13,2026-01-13 23:05:00,도리당 송파점,이병두 과장,byoungd201@kakao.com,29,867400.0,40090.0,29910.344828,1182021.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,2026-01-13,2026-01-13 23:50:00,도리당 역삼점,심성준 이사,simjeong00@kakao.com,129,3640800.0,96459.0,28223.255814,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,2026-01-13,2026-01-13 22:35:00,도리당 청라점,이병두 과장,byoungd201@kakao.com,38,1182600.0,44440.0,31121.052632,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged_df = merged_df.merge(
    toorder_review_df,
    how="left",
    left_on=["order_daily", "매장명"],
    right_on=["date", "stores_name"])
merged_df.drop(columns=["date", "stores_name"], inplace=True)

In [ ]:
merged_df.head()

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,조리시간준수율,조리시간준수율_순위비율,주문접수율,주문접수율_순위비율,최근별점,영업임시중지변경_변경_cnt,전체_주문수,전체_리뷰수,전체_답변완료수,전체_평균별점
0,2025-12-01,2025-12-01 20:21:00,도리당 교대점,심성준 이사,simjeong00@kakao.com,12,327100.0,15540.0,27258.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-12-01,2025-12-01 23:42:37,도리당 미사점,김대진 팀장,sanbogaja81@kakao.com,63,4492100.0,163890.0,71303.174603,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-12-01,2025-12-01 23:43:21,도리당 송파점,이병두 과장,byoungd201@kakao.com,53,1380300.0,72010.0,26043.396226,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-12-01,2025-12-01 23:45:38,도리당 운정점,김덕기 과장,kdk1402@kakao.com,39,1106700.0,42840.0,28376.923077,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-12-01,2025-12-01 23:33:00,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,48,1213600.0,23670.0,25283.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [5]:
df = pd.read_csv(r"C:\Local_DB\영업관리부_DB\sales_daily_orders.csv")
df.head()


C:\Users\민준\AppData\Local\Temp\ipykernel_1120\3631136942.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Local_DB\영업관리부_DB\sales_daily_orders.csv")


,platform,order_date,store_id,store_name,stores,store_names,ad_id,ad_product,order_id,sub_order_id,...,매장명,담당자,상세주소,광역,시군구,읍면동,employee_collected_at,email,_row_hash,실오픈일
0,배민,2025-12-31 20:22:45,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T2950001I4O5,T2950001I4O5_1_12d7,...,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-14 05:34:37,simjeong00@kakao.com,12d7c9b9df2e,2024-06-07
1,배민,2025-12-31 20:22:45,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T2950001I4O5,T2950001I4O5_2_8cbf,...,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-14 05:34:37,simjeong00@kakao.com,8cbf1806a289,2024-06-07
2,배민,2025-12-31 20:22:45,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T2950001I4O5,T2950001I4O5_3_cbbc,...,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-14 05:34:37,simjeong00@kakao.com,cbbc554b68cd,2024-06-07
3,배민,2025-12-31 20:22:45,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T2950001I4O5,T2950001I4O5_4_67fd,...,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-14 05:34:37,simjeong00@kakao.com,67fd90f7e3ac,2024-06-07
4,배민,2025-12-31 20:22:45,NaN,[음식배달] 닭도리탕 전문 도리당 교대점,도리당 교대점,도리당 교대점,12460879.0,배민1플러스 (배민클럽),T2950001I4O5,T2950001I4O5_5_f3cc,...,도리당 교대점,심성준 이사,"서울시 서초구 법원로2길 7-4, 지하2층 210호 중 10호(서초동, 동용빌딩)",서울,서초구,법원로2길,2026-01-14 05:34:37,simjeong00@kakao.com,f3ccca3836de,2024-06-07


In [11]:
alert_df = pd.read_parquet(r"C:\Local_DB\temp\sales_daily_orders_20260114_084533.parquet")
print(f"행 수: {len(alert_df)}, 열 수: {len(alert_df.columns)}")
alert_df

행 수: 298, 열 수: 55


,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,일별_전체_평균,vs일별전체_비율,fee_ad_배민,fee_ad_쿠팡,settlement_amount_배민,settlement_amount_쿠팡,total_amount_배민,total_amount_쿠팡,total_order_count_배민,total_order_count_쿠팡
0,2025-12-31,2025-12-31 18:48:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,6,171200.0,12910.0,28533.333333,NaN,...,1579543.0,10.8,0.0,12910.0,0.0,105517.0,0.0,171200.0,0,6
1,2026-01-01,2026-01-01 19:58:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,7,224300.0,18495.0,32042.857143,NaN,...,1843271.0,12.2,0.0,18495.0,0.0,143391.0,0.0,224300.0,0,7
2,2026-01-02,2026-01-02 21:20:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,5,95600.0,7200.0,19120.000000,NaN,...,1376062.0,6.9,0.0,7200.0,0.0,40216.0,0.0,95600.0,0,5
3,2026-01-03,2026-01-03 11:19:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,1,17600.0,0.0,17600.000000,NaN,...,1412462.0,1.2,0.0,0.0,0.0,7901.0,0.0,17600.0,0,1
4,2026-01-13,2026-01-13 21:13:23,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,22,546100.0,25345.0,24822.727273,NaN,...,1816329.0,30.1,4000.0,21345.0,198919.0,173831.0,279600.0,266500.0,13,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,2026-01-10,2026-01-10 23:56:23,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,64,2164800.0,41660.0,33825.000000,1757957.14,...,2222743.0,97.4,0.0,41660.0,1213095.0,379310.0,1590300.0,574500.0,48,16
294,2026-01-11,2026-01-11 23:03:33,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,99,3105200.0,41910.0,31365.656566,1844992.86,...,2451950.0,126.6,0.0,41910.0,1987830.0,423407.0,2446100.0,659100.0,77,22
295,2026-01-12,2026-01-12 23:23:54,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,41,1294900.0,12070.0,31582.926829,1867800.00,...,2019570.0,64.1,0.0,12070.0,919807.0,113090.0,1119300.0,175600.0,36,5
296,2026-01-13,2026-01-13 23:46:11,도리당 평택서정점,황대성 대리,gjddkemf@kakao.com,61,1767600.0,24720.0,28977.049180,1895492.86,...,1816329.0,97.3,0.0,24720.0,1125162.0,193171.0,1442600.0,325000.0,48,13


In [1]:
import pandas as pd

In [48]:
pd.read_parquet(r"C:\Local_DB\temp\joined_orders_now_toorder_path_20260115.parquet").head()

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,주문접수시간_순위비율,조리시간준수율,조리시간준수율_순위비율,주문접수율,주문접수율_순위비율,최근별점,전체_주문수,전체_리뷰수,전체_답변완료수,전체_평균별점
0,2025-12-31,2025-12-31 18:48:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,6,171200.0,12910.0,28533.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20.0,5.0,5.0,1.428571
1,2026-01-01,2026-01-01 19:58:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,7,224300.0,18495.0,32042.857143,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,28.0,7.0,7.0,1.392857
2,2026-01-02,2026-01-02 21:20:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,5,95600.0,7200.0,19120.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.0,9.0,9.0,1.428571
3,2026-01-03,2026-01-03 11:19:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,1,17600.0,0.0,17600.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,20.0,9.0,9.0,1.404286
4,2026-01-13,2026-01-13 21:13:23,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,22,546100.0,25345.0,24822.727273,NaN,...,0.4,1.0,0.1,0.987,0.4,4.9,22.0,0.0,0.0,0.000000


merged_df = merged_df.merge(history_pivot,
                how="left",
                left_on=["order_daily", "매장명"],
                right_on=["change_date", "stores_name"]
)
merged_df.drop(columns=["change_date", "stores_name"], inplace=True)
merged_df

In [55]:
pd.read_parquet(r"C:\Local_DB\temp\joined_orders_now_toorder_history_path_20260115.parquet").head()

,order_daily,order_date_with_time,매장명,담당자,email,total_order_count,total_amount,fee_ad,ARPU,ma_14,...,조리시간준수율,조리시간준수율_순위비율,주문접수율,주문접수율_순위비율,최근별점,전체_주문수,전체_리뷰수,전체_답변완료수,전체_평균별점,영업임시중지 변경
0,2025-12-31,2025-12-31 18:48:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,6,171200.0,12910.0,28533.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,20.0,5.0,5.0,1.428571,NaN
1,2026-01-01,2026-01-01 19:58:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,7,224300.0,18495.0,32042.857143,NaN,...,NaN,NaN,NaN,NaN,NaN,28.0,7.0,7.0,1.392857,NaN
2,2026-01-02,2026-01-02 21:20:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,5,95600.0,7200.0,19120.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,17.0,9.0,9.0,1.428571,NaN
3,2026-01-03,2026-01-03 11:19:00,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,1,17600.0,0.0,17600.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,20.0,9.0,9.0,1.404286,NaN
4,2026-01-13,2026-01-13 21:13:23,도리당 가락점,김대진 팀장,sanbogaja81@kakao.com,22,546100.0,25345.0,24822.727273,NaN,...,NaN,NaN,NaN,NaN,NaN,22.0,0.0,0.0,0.000000,NaN
